In [25]:
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.simplefilter('ignore')


#讀取訓練與測試資料
df_train = pd.read_csv('/Users/max/Downloads/titanic_train.csv')
df_test = pd.read_csv('/Users/max/Downloads/titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [26]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


In [27]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.780004837244799


,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


## 計數編碼 ( Counting ) 
    * 如果類別的⽬標均價與類別筆數呈正相關 ( 或負相關 )，也可以將筆數本⾝當成特徵

In [28]:
# 加上 'Cabin' 欄位的計數編碼
df_count = df.groupby('Cabin')['Name'].agg('size').reset_index()
df_count.columns = ['Cabin', 'Cabin_Count']
df = pd.merge(df, df_count, on='Cabin', how='left')
df.head()

,Name,Sex,Ticket,Cabin,Embarked,Cabin_Count
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S,1014
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,2
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S,1014
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,2
4,"Allen, Mr. William Henry",male,373450,None,S,1014


In [29]:
# 標籤編碼 + 'Cabin'計數編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for col in object_features:
    df_temp[col] = LabelEncoder().fit_transform(df[col])
df_temp['Cabin_Count'] = df['Cabin_Count']
train_X = df_temp[:train_num]
clf = LogisticRegression()
print(cross_val_score(clf, train_X, train_Y, cv=5, n_jobs=-1).mean())
df_temp.head()

0.7856230275549181


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Count
0,155,1,720,185,3,1014
1,286,0,816,106,0,2
2,523,0,914,185,3,1014
3,422,0,65,70,3,2
4,22,1,649,185,3,1014


## 特徵雜湊 ( Feature Hash ) 
    * 特徵雜湊是⼀種折衷⽅案
    * 將類別由雜湊函數定應到⼀組數字
    * 調整雜湊函數對應值的數量
    * 在計算空間/時間與鑑別度間取折衷
    * 也提⾼了訊息密度, 減少無⽤的標籤

In [30]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for col in object_features:
    df_temp[col] = LabelEncoder().fit_transform(df[col])
df_temp['Cabin_Hash'] = df['Cabin'].map(lambda x: hash(x) % 10)
train_X = df_temp[:train_num]
clf = LogisticRegression()
print(cross_val_score(clf, train_X, train_Y, cv=5, n_jobs=-1).mean())
df_temp.head()

0.774386788789366


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Hash
0,155,1,720,185,3,4
1,286,0,816,106,0,3
2,523,0,914,185,3,4
3,422,0,65,70,3,8
4,22,1,649,185,3,4


In [32]:
# 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for col in object_features:
    df_temp[col] = LabelEncoder().fit_transform(df[col])
df_temp['Cabin_Hash'] = df['Cabin'].map(lambda x: hash(x) % 10)
df_temp['Cabin_Count'] = df['Cabin_Count']
train_X = df_temp[:train_num]
clf = LogisticRegression()
print(cross_val_score(clf, train_X, train_Y, cv=5, n_jobs=-1).mean())
df_temp.head()

0.7800238103090765


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Hash,Cabin_Count
0,155,1,720,185,3,4,1014
1,286,0,816,106,0,3,2
2,523,0,914,185,3,4,1014
3,422,0,65,70,3,8,2
4,22,1,649,185,3,4,1014
